# now that we have reached here, we want to see the value of the stock of the company at the following time periods:¶
1)The Price of the stock right before the news was published

2)The Price of the stock when the news was published

3)The Price one day after the news was published

4)The Price one week after the news was published

The objective of the this project is to see and understand the violatility of the market and news affect them. Can news published by reuters, which is one of the biggest and most trusted news wire service affect the company stock price in any way ?

For this, we will use playwright and beautifulSoup. From the nasdaq website, we will automate the browser to go to the company historical data and look for the past one year data. we will then build a function that collects the dat from the table from every page and loops till the last page.once we have the data, we can import it in pandas and make a dataframe.

In [1]:
from playwright.async_api import async_playwright

In [2]:
playwright = await async_playwright().start()

In [3]:
browser = await playwright.chromium.launch(headless = False)

In [4]:
browser.new_page()

<coroutine object Browser.new_page at 0x105c39000>

In [5]:
page = await browser.new_page()

In [7]:
await page.goto("https://www.nasdaq.com/market-activity/stocks/tsla/historical")

<Response url='https://www.nasdaq.com/market-activity/stocks/tsla/historical' request=<Request url='https://www.nasdaq.com/market-activity/stocks/tsla/historical' method='GET'>>

In [8]:
await page.content()

'<!DOCTYPE html><html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# " class=" js"><head><script charset="UTF-8" type="text/javascript" async="async" src="https://cdn.taboola.com/libtrc/userx.20230212-22_b4-DEV-112567-trecs-interop-network-milestone2-short-946b6b4b828.es6.js"></script><script charset="UTF-8" type="text/javascript" async="async" src="https://cdn.taboola.com/libtrc/cta-component.20230212-22_b4-DEV-112567-trecs-interop-network-milestone2-short-946b6b4b828.es6.js"></script><script charset="UTF-8" type="text/javascript" async="async" src="https://cdn.taboola.com/libtrc/feed-card-placeholder.20230212-22_b4-DEV-112567-trecs-interop-networ

In [9]:
await page.locator("xpath=/html/body/div[3]/div/main/div[2]/div[4]/div[3]/div/div[1]/div/div[1]/div[3]/div/div/div/button[2]").click()

In [10]:
import asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import time
# Initialize an empty list to store the data
stock_data = []

async def scrape_stock_data(page, soup):
    # Extract the data from the table
    await page.wait_for_selector("table.historical-data__table")
    table = soup.find("table", class_="historical-data__table")
    rows = table.find_all("tr")
    for row in rows:
        date_cell = row.find("th", class_="historical-data__cell")
        date = date_cell.text if date_cell is not None else ""
        cells = row.find_all("td", class_="historical-data__cell")
        row_data = [cell.text for cell in cells]
        stock_data.append([date] + row_data)

async def main():
    # Create a Playwright instance
    async with async_playwright() as playwright:
        # Start a browser
        browser = playwright.chromium
        # Create a new page
        page = await browser.new_page()
        
        await page.wait_for_selector("table.historical-data__table")
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser") 
        await scrape_stock_data(page, soup) 
        
        
html = await page.content()
soup = BeautifulSoup(html, "html.parser")        
current_page = int(soup.select_one("button.pagination__page")["data-page"])
await scrape_stock_data(page, soup) 

# Loop until we reach the last page
while True:
    # Check if there is a "Next" button
    next_button = page.locator(f"button.pagination__page[data-page='{current_page + 1}']")
    try:
        # Click the "Next" button to retrieve the next page
        await next_button.click()
        # Wait for the page to load
        await page.wait_for_selector("table.historical-data__table")
        time.sleep(3)
        # Extract the data from the current page
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        await scrape_stock_data(page, soup)
        # Increment the current page number
        current_page += 1
    except:
        # We have reached the last page, so break the loop
        break

async def main():
    # Create a Playwright instance
    async with async_playwright() as playwright:
        # Start a browser
        browser = playwright.chromium
        # Create a new page
        page = await browser.new_page()
        # Scrape the stock data
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        await scrape_stock_data(page, soup)


# Print the scraped stock data
print(stock_data)

[[''], ['02/14/2023', '$209.25', '216,455,700', '$191.94', '$209.82', '$189.44'], ['02/13/2023', '$194.64', '172,475,500', '$194.415', '$196.3', '$187.61'], ['02/10/2023', '$196.89', '204,754,100', '$202.225', '$206.2', '$192.89'], ['02/09/2023', '$207.32', '215,431,400', '$207.775', '$214', '$204.77'], ['02/08/2023', '$201.29', '180,673,600', '$196.1', '$203', '$194.31'], ['02/07/2023', '$196.81', '186,010,300', '$196.43', '$197.5', '$189.55'], ['02/06/2023', '$194.76', '186,188,100', '$193.01', '$198.17', '$189.92'], ['02/03/2023', '$189.98', '232,662,000', '$183.95', '$199', '$183.69'], ['02/02/2023', '$188.27', '217,448,300', '$187.325', '$196.7501', '$182.61'], ['02/01/2023', '$181.41', '213,806,300', '$173.89', '$183.805', '$169.93'], ['01/31/2023', '$173.22', '196,813,500', '$164.57', '$174.3', '$162.78'], ['01/30/2023', '$166.66', '230,878,800', '$178.05', '$179.77', '$166.5'], ['01/27/2023', '$177.9', '306,590,600', '$162.43', '$180.68', '$161.17'], ['01/26/2023', '$160.27', '

In [11]:
stock_data

[[''],
 ['02/14/2023', '$209.25', '216,455,700', '$191.94', '$209.82', '$189.44'],
 ['02/13/2023', '$194.64', '172,475,500', '$194.415', '$196.3', '$187.61'],
 ['02/10/2023', '$196.89', '204,754,100', '$202.225', '$206.2', '$192.89'],
 ['02/09/2023', '$207.32', '215,431,400', '$207.775', '$214', '$204.77'],
 ['02/08/2023', '$201.29', '180,673,600', '$196.1', '$203', '$194.31'],
 ['02/07/2023', '$196.81', '186,010,300', '$196.43', '$197.5', '$189.55'],
 ['02/06/2023', '$194.76', '186,188,100', '$193.01', '$198.17', '$189.92'],
 ['02/03/2023', '$189.98', '232,662,000', '$183.95', '$199', '$183.69'],
 ['02/02/2023', '$188.27', '217,448,300', '$187.325', '$196.7501', '$182.61'],
 ['02/01/2023', '$181.41', '213,806,300', '$173.89', '$183.805', '$169.93'],
 ['01/31/2023', '$173.22', '196,813,500', '$164.57', '$174.3', '$162.78'],
 ['01/30/2023', '$166.66', '230,878,800', '$178.05', '$179.77', '$166.5'],
 ['01/27/2023', '$177.9', '306,590,600', '$162.43', '$180.68', '$161.17'],
 ['01/26/2023'

In [12]:
import pandas as pd

# Create a DataFrame from the stock_data list
df = pd.DataFrame(stock_data, columns=["Date", "Close/Last", "Volume", "Open", "High", "Low"])

In [30]:
df

In [31]:
pd.options.display.max_rows = None

In [32]:
df

In [33]:
df = df.dropna()

AttributeError: 'NoneType' object has no attribute 'dropna'

In [34]:
df

In [35]:
df.to_csv("nasdaqtesla_stock_data.csv")

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [36]:
df

In [37]:
stock_data

[[''],
 ['02/14/2023', '$209.25', '216,455,700', '$191.94', '$209.82', '$189.44'],
 ['02/13/2023', '$194.64', '172,475,500', '$194.415', '$196.3', '$187.61'],
 ['02/10/2023', '$196.89', '204,754,100', '$202.225', '$206.2', '$192.89'],
 ['02/09/2023', '$207.32', '215,431,400', '$207.775', '$214', '$204.77'],
 ['02/08/2023', '$201.29', '180,673,600', '$196.1', '$203', '$194.31'],
 ['02/07/2023', '$196.81', '186,010,300', '$196.43', '$197.5', '$189.55'],
 ['02/06/2023', '$194.76', '186,188,100', '$193.01', '$198.17', '$189.92'],
 ['02/03/2023', '$189.98', '232,662,000', '$183.95', '$199', '$183.69'],
 ['02/02/2023', '$188.27', '217,448,300', '$187.325', '$196.7501', '$182.61'],
 ['02/01/2023', '$181.41', '213,806,300', '$173.89', '$183.805', '$169.93'],
 ['01/31/2023', '$173.22', '196,813,500', '$164.57', '$174.3', '$162.78'],
 ['01/30/2023', '$166.66', '230,878,800', '$178.05', '$179.77', '$166.5'],
 ['01/27/2023', '$177.9', '306,590,600', '$162.43', '$180.68', '$161.17'],
 ['01/26/2023'

In [38]:
import pandas as pd

# Create a DataFrame from the stock_data list
df_stock_data = pd.DataFrame(stock_data, columns=["Date", "Close/Last", "Volume", "Open", "High", "Low"])

In [39]:
df_stock_data

Date Close/Last       Volume       Open       High        Low
0                     None         None       None       None       None
1    02/14/2023    $209.25  216,455,700    $191.94    $209.82    $189.44
2    02/13/2023    $194.64  172,475,500   $194.415     $196.3    $187.61
3    02/10/2023    $196.89  204,754,100   $202.225     $206.2    $192.89
4    02/09/2023    $207.32  215,431,400   $207.775       $214    $204.77
5    02/08/2023    $201.29  180,673,600     $196.1       $203    $194.31
6    02/07/2023    $196.81  186,010,300    $196.43     $197.5    $189.55
7    02/06/2023    $194.76  186,188,100    $193.01    $198.17    $189.92
8    02/03/2023    $189.98  232,662,000    $183.95       $199    $183.69
9    02/02/2023    $188.27  217,448,300   $187.325  $196.7501    $182.61
10   02/01/2023    $181.41  213,806,300    $173.89   $183.805    $169.93
11   01/31/2023    $173.22  196,813,500    $164.57     $174.3    $162.78
12   01/30/2023    $166.66  230,878,800    $178.05    $179.77     $166.5
13   01/27/2023     $177.9  306,590,600    $162.43    $180.68    $161.17
14   01/26/2023    $160.27  234,815,100   $159.965    $161.42    $154.76
15   01/25/2023    $144.43  192,734,300   $141.905    $146.41    $138.07
16   01/24/2023    $143.89  158,699,100       $143     $146.5     $141.1
17   01/23/2023    $143.75  203,119,200    $135.87  $145.3793    $134.27
18   01/20/2023    $133.42  138,858,100    $128.68    $133.51  $127.3466
19                    None         None       None       None       None
20   01/19/2023    $127.17  170,291,900    $127.26    $129.99  $124.3082
21   01/18/2023    $128.78  195,680,300   $136.555    $136.68    $127.01
22   01/17/2023    $131.49  186,477,000   $125.695     $131.7    $125.02
23   01/13/2023     $122.4  180,714,100    $116.55    $122.63     $115.6
24   01/12/2023    $123.56  169,400,900    $122.56  $124.1311       $117
25   01/11/2023    $123.22  183,810,800    $122.09    $125.95    $120.51
26   01/10/2023    $118.85  167,642,500    $121.07    $122.76    $114.92
27   01/09/2023    $119.77  190,284,000    $118.96    $123.52    $117.11
28   01/06/2023    $113.06  220,911,100       $103    $114.39    $101.81
29   01/05/2023    $110.34  157,986,300    $110.51    $111.75  $107.1601
30   01/04/2023    $113.64  180,389,000    $109.11    $114.59    $107.52
31   01/03/2023     $108.1  231,402,800    $118.47     $118.8    $104.64
32   12/30/2022    $123.18  157,777,300    $119.95    $124.48    $119.75
33   12/29/2022    $121.82  221,923,300   $120.385    $123.57   $117.495
34   12/28/2022    $112.71  221,070,500    $110.35    $116.27    $108.24
35   12/27/2022     $109.1  208,643,400   $117.495    $119.67    $108.76
36   12/23/2022    $123.15  166,989,700    $126.37  $128.6173    $121.02
37   12/22/2022    $125.35  210,090,300       $136    $136.63    $122.26
38                    None         None       None       None       None
39   12/21/2022    $137.57  145,417,400   $139.335    $141.26    $135.89
40   12/20/2022     $137.8  159,563,300    $146.05    $148.47    $137.66
41   12/19/2022    $149.87  139,390,600       $154    $155.25    $145.82
42   12/16/2022    $150.23  139,032,200   $159.635    $160.99    $150.04
43   12/15/2022    $157.67  122,334,500    $153.44  $160.9299    $153.28
44   12/14/2022     $156.8  140,682,300    $159.25    $161.62    $155.31
45   12/13/2022    $160.95  175,862,700    $174.87    $175.05    $156.91
46   12/12/2022    $167.82  109,794,500     $176.1    $177.37    $167.52
47   12/09/2022    $179.05  104,872,300    $173.84     $182.5    $173.36
48   12/08/2022    $173.44   97,624,490     $172.2     $175.2    $169.06
49   12/07/2022    $174.04   84,213,280    $175.03    $179.38   $172.218
50   12/06/2022    $179.82   92,150,820   $181.215    $183.65    $175.33
51   12/05/2022    $182.45   93,122,670    $189.44    $191.27    $180.55
52   12/02/2022    $194.86   73,645,920    $191.78    $196.25    $191.11
53   12/01/2022     $194.7   80,046,210    $197.08    $198.92     $19

In [41]:
df = df_stock_data.dropna()

In [42]:
df

Date Close/Last       Volume       Open       High        Low
1    02/14/2023    $209.25  216,455,700    $191.94    $209.82    $189.44
2    02/13/2023    $194.64  172,475,500   $194.415     $196.3    $187.61
3    02/10/2023    $196.89  204,754,100   $202.225     $206.2    $192.89
4    02/09/2023    $207.32  215,431,400   $207.775       $214    $204.77
5    02/08/2023    $201.29  180,673,600     $196.1       $203    $194.31
6    02/07/2023    $196.81  186,010,300    $196.43     $197.5    $189.55
7    02/06/2023    $194.76  186,188,100    $193.01    $198.17    $189.92
8    02/03/2023    $189.98  232,662,000    $183.95       $199    $183.69
9    02/02/2023    $188.27  217,448,300   $187.325  $196.7501    $182.61
10   02/01/2023    $181.41  213,806,300    $173.89   $183.805    $169.93
11   01/31/2023    $173.22  196,813,500    $164.57     $174.3    $162.78
12   01/30/2023    $166.66  230,878,800    $178.05    $179.77     $166.5
13   01/27/2023     $177.9  306,590,600    $162.43    $180.68    $161.17
14   01/26/2023    $160.27  234,815,100   $159.965    $161.42    $154.76
15   01/25/2023    $144.43  192,734,300   $141.905    $146.41    $138.07
16   01/24/2023    $143.89  158,699,100       $143     $146.5     $141.1
17   01/23/2023    $143.75  203,119,200    $135.87  $145.3793    $134.27
18   01/20/2023    $133.42  138,858,100    $128.68    $133.51  $127.3466
20   01/19/2023    $127.17  170,291,900    $127.26    $129.99  $124.3082
21   01/18/2023    $128.78  195,680,300   $136.555    $136.68    $127.01
22   01/17/2023    $131.49  186,477,000   $125.695     $131.7    $125.02
23   01/13/2023     $122.4  180,714,100    $116.55    $122.63     $115.6
24   01/12/2023    $123.56  169,400,900    $122.56  $124.1311       $117
25   01/11/2023    $123.22  183,810,800    $122.09    $125.95    $120.51
26   01/10/2023    $118.85  167,642,500    $121.07    $122.76    $114.92
27   01/09/2023    $119.77  190,284,000    $118.96    $123.52    $117.11
28   01/06/2023    $113.06  220,911,100       $103    $114.39    $101.81
29   01/05/2023    $110.34  157,986,300    $110.51    $111.75  $107.1601
30   01/04/2023    $113.64  180,389,000    $109.11    $114.59    $107.52
31   01/03/2023     $108.1  231,402,800    $118.47     $118.8    $104.64
32   12/30/2022    $123.18  157,777,300    $119.95    $124.48    $119.75
33   12/29/2022    $121.82  221,923,300   $120.385    $123.57   $117.495
34   12/28/2022    $112.71  221,070,500    $110.35    $116.27    $108.24
35   12/27/2022     $109.1  208,643,400   $117.495    $119.67    $108.76
36   12/23/2022    $123.15  166,989,700    $126.37  $128.6173    $121.02
37   12/22/2022    $125.35  210,090,300       $136    $136.63    $122.26
39   12/21/2022    $137.57  145,417,400   $139.335    $141.26    $135.89
40   12/20/2022     $137.8  159,563,300    $146.05    $148.47    $137.66
41   12/19/2022    $149.87  139,390,600       $154    $155.25    $145.82
42   12/16/2022    $150.23  139,032,200   $159.635    $160.99    $150.04
43   12/15/2022    $157.67  122,334,500    $153.44  $160.9299    $153.28
44   12/14/2022     $156.8  140,682,300    $159.25    $161.62    $155.31
45   12/13/2022    $160.95  175,862,700    $174.87    $175.05    $156.91
46   12/12/2022    $167.82  109,794,500     $176.1    $177.37    $167.52
47   12/09/2022    $179.05  104,872,300    $173.84     $182.5    $173.36
48   12/08/2022    $173.44   97,624,490     $172.2     $175.2    $169.06
49   12/07/2022    $174.04   84,213,280    $175.03    $179.38   $172.218
50   12/06/2022    $179.82   92,150,820   $181.215    $183.65    $175.33
51   12/05/2022    $182.45   93,122,670    $189.44    $191.27    $180.55
52   12/02/2022    $194.86   73,645,920    $191.78    $196.25    $191.11
53   12/01/2022     $194.7   80,046,210    $197.08    $198.92     $191.8
54   11/30/2022     $194.7  109,186,400    $182.43    $194.76  $180.6301
55   11/29/2022    $180.83   83,357,110    $184.99    $186.38    $178.75
56   11/28/2022    $182.92   93,038,150    $179.96     $188.5       $

In [43]:
df.to_csv("nasdaq_tesla_stock_data.csv")